## Building a dataset using OSC

Here you can use Processing to send images, and create a dataset

In [ ]:
#install a required library to use OSC protocol
!pip install https://github.com/attwad/python-osc/archive/master.zip
!pip install matplotlib

In [ ]:

#location where to store your dataset
dataset_dir="datasets/actions/"
#labels of the dataset
labels=["None","Left","Right","Head"]


In [ ]:
import OSCHelper
server=OSCHelper.createServer(9000)
client=OSCHelper.createClient(4200)

In [ ]:

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
#from scipy import misc
import io
from PIL import Image
import os


active_label=0
img_idx=0
    
def make_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

make_dir(dataset_dir)

def onOSC_Image(*args):
    print("image")
    
    global active_label
    global img_idx
    if active_label>=len(labels):
        return
    
    width=args[1]
    height=args[2]
    jpgdata=args[3]
    file_jpgdata = io.BytesIO(jpgdata)
    img = Image.open(file_jpgdata).convert("L")
    #img=img.convert("L")
    
    img.save(os.path.join(dataset_dir,labels[active_label],"img_{0}.jpg".format(img_idx)))
    img_idx=img_idx+1
    
    img=np.array(img)
    #args=np.array()
    #args=args/255.0
    #img=np.reshape(args,(width,height))
    #plt.imshow(img)
    #plt.title("Showing image from: {0}x{1}".format(width,height)) 
    #plt.show()

def send_current_label():
    client.send_message("/output/label",[active_label,labels[active_label]])
    
def make_label_dir():
    make_dir(os.path.join(dataset_dir,labels[active_label]))
    
def onOSC_Next_Image(*args):
    print("next")
    global active_label
    global img_idx
    active_label=active_label+1
    if active_label>=len(labels):
        client.send_message("/output/done")
        return
    img_idx=0
    make_label_dir()
    send_current_label()
    
    
def onOSC_Reset_Image(*args):
    print("reset")
    global active_label
    global img_idx
    active_label=0
    img_idx=0
    send_current_label()
    
server.addMsgHandler( "/inputs/image", onOSC_Image )
server.addMsgHandler( "/inputs/next", onOSC_Next_Image )
server.addMsgHandler( "/inputs/reset", onOSC_Reset_Image )


In [ ]:
make_label_dir()
send_current_label()
OSCHelper.start_server(server)


In [ ]:

server.close()